# Reconhecimento de flores e árvores

## Autora
* Isabella Carneiro Godinho de Morais Sá
* Matrícula: 18/0019066
* Github: isabellacgmsa

## Objetivo
O propósito do presente artigo é demonstrar o conhecimento adquirido através da segunda lição, que ensina a usar matriz de confusão, ImageClassifierCleaner e deploy.

## Dependências

A instalação das dependências é um passo fundamental para garantir que o processo possa ser executado com sucesso e que os resultados sejam precisos e confiáveis. Além disso, a escolha das bibliotecas e ferramentas adequadas é essencial para garantir que o processo seja executado de maneira eficiente e eficaz, maximizando os recursos disponíveis e minimizando o tempo necessário para a execução do processo.
Assim sendo, o primeiro passo que foi tomado para iniciar o projeto foi a instalação das dependências necessárias para que o processo pudesse ser executado de maneira adequada.
Para tanto, foram utilizados os seguintes comandos para a instalação das dependências necessárias:

```pip install -U duckduckgo_search```

```pip install fastai"```

Vale ressaltar que, antes de iniciar o processo, é necessário fazer a instalação do PyTorch, que é uma biblioteca de aprendizado de máquina de código aberto amplamente utilizada para tarefas como visão computacional e processamento de linguagem natural.

## Baixando imagens
Para fazer a instalação das imagens foi utilizado o ddg_images, como mostra o código a seguir:  
```
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term: str, max_images=30):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')
    
```

O primeiro teste será feito baixando apenas uma imagem de flor:

```
urls = search_images('flower', max_images=1)
urls[2]
```

O download será realizado utilizando a biblioteca fastdownload:

```
from fastdownload import download_url
dest = 'oneflower.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)
```

![texto](https://s3.static.brasilescola.uol.com.br/be/2020/12/girassol.jpg)

Ao constatar que está funcionando todo o processo é possível fazer o download de dados mais numerosos:
```
searches = 'tree', 'flower'
path = Path('tree_flower')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} tree'))
    sleep(10)
    download_images(dest, urls=search_images(f'{o} flower'))
    sleep(10)
    resize_images(path/o, max_size=400, dest= path/o)
```

Foi utilizado o sleep para que não haja problemas no tráfego de informações.

Essas imagens serão salvas em uma pasta cujo caminho está definido no path.

## Treinamento
### Passo 1
Antes de iniciar o treinamento do modelo de fato é necessário garantir que não haja imagens com erro para evitar futuros problemas:

```
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)
```
### Passo 2

Para treinar modelos de aprendizado de máquina com conjuntos de dados volumosos, é necessário dividir os dados em lotes menores para evitar sobrecarregar a memória RAM do computador. O Dataloader é uma classe que oferece essa funcionalidade de forma eficiente. Ele carrega e processa os dados em lotes menores, tornando o processo de treinamento escalável.

O fastai é um framework de aprendizado de máquina que permite o uso do datablock, uma ferramenta que facilita a personalização do processamento e organização dos dados em lotes para treinamento do modelo. Com o datablock, é possível definir etapas como leitura, pré-processamento e organização dos dados de forma mais simples. Isso permite que os usuários criem fluxos de trabalho personalizados e experimentem diferentes abordagens para melhorar a precisão do modelo.

```
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=6)
```
![texto](https://i.ibb.co/T4XzYk1/flores.png)

Foi especificado que há o ImageBlock, que é uma classe do fastai que permite a criação de blocos para processar as imagens no modelo, e o CategoryBlock que cria blocos para processar variáveis categóricas, nesse caso árvore ou flor. Nesse trecho também especifica o tamanho que as imagens terão, já que precisa ser o mesmo para todas.

### Passo 3
Após criar o Datablock, o modelo é treinado com a ajuda do objeto "Learner". Ele é configurado com três argumentos: o primeiro é um DataLoader (dls) que contém o conjunto de dados de treinamento do modelo; o segundo é a arquitetura ResNet18 que será usada como base para o modelo; e o terceiro é a métrica usada para avaliar a performance do modelo, que neste caso é a taxa de erro.

Para realizar o ajuste fino do modelo, é utilizado o método "fine_tune()". Esse método realiza um treinamento adicional do modelo usando os pesos pré-treinados da ResNet18 como ponto de partida.
```
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(8)
```

### Passo 4
Utilizaremos a matriz de confusão, que é uma ferramenta muito utilizada em aprendizado de máquina para avaliar o desempenho de um modelo. Ela apresenta a frequência de acertos e erros do modelo em relação às classes que estão sendo avaliadas.
Com ela é possível obter diversas métricas importantes, como a acurácia, a precisão, o recall, a taxa de falsos positivos e a taxa de falsos negativos. Com base nessas métricas, é possível avaliar o desempenho do modelo e fazer ajustes que permitam melhorar sua precisão e acurácia.

```
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()
```
![imagem](https://i.ibb.co/Xj1W9cs/matriz.png)

### Passo 5
Foi utilizada também a função plot_top_losses para identificar as imagens que mais contribuem para o erro de um modelo. Essa função retorna as imagens que apresentam os maiores valores de erro em relação ao valor de loss calculado para cada uma das imagens em um conjunto de dados.

Ao analisar as imagens que apresentam os maiores valores de erro, é possível identificar padrões e características que podem estar contribuindo para a performance insatisfatória do modelo, possibilitando ajustes e melhorias. Além disso, a análise dessas imagens também pode ajudar a identificar erros na rotulagem dos dados, que podem estar comprometendo a acurácia do modelo.

A utilização da função plot_top_losses é uma etapa importante no processo de avaliação e ajuste de um modelo de aprendizado de máquina. A partir da identificação das imagens que mais contribuem para o erro, é possível fazer ajustes que permitam melhorar a precisão e a acurácia do modelo, tornando-o mais confiável e eficaz.

```
interp.plot_top_losses(5, nrows=1)

```

![image](https://i.ibb.co/vZRBStK/download.png)


### Passo 6
Para garantir a acurácia das imagens testadas usa-se o ImageClassifierCleaner que permite a remoção de imagens incorretamente rotuladas em um conjunto de dados de treinamento.

```
cleaner = ImageClassifierCleaner(learn)
cleaner
```

Usamos o seguinte código para remover imagens incorretas e reorganizar as imagens corretamente rotuladas em um diretório específico, facilitando assim o processo de limpeza e preparação dos dados para o treinamento do modelo de aprendizado de máquina.

```
for idx in cleaner.delete(): cleaner.fns[idx].unlink()
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)
```

### Passo 7
Para realizar o deploy utiliza-se a função export para criar um arquivo com todas as informações e configurações necessárias:

```
learn.export('model.pkl')
```

O deploy é realizado utilizando um space na Hugging Face criando um commit do arquivo que foi gerado:

![imagem](https://i.ibb.co/7Kg6Scm/download-1.png)


## Conclusão
Em conclusão, o desenvolvimento e deploy de um modelo de aprendizado de máquina capaz de identificar flores e árvores é um processo complexo, que envolve várias etapas, desde a preparação e limpeza dos dados, passando pela escolha do modelo e dos parâmetros, até a avaliação e ajuste do modelo.
O Hugging Face é simples de entender como subir o modelo e o fastai possui várias ferramentas que auxiliam e deixam mais simples desenvolver.
Há alguns ajustes a serem feitos no modelo para reduzir erros e, consequentemente, aumentar sua acurácia.



